# Dependences

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

# Model

In [2]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000009 seconds (5 allocations: 208 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [3]:
Par1 = Cubic(0.001,2.0,3.0)
Par2 = ContractilePar(0.001)
agg_size = 15

15

# Aggregate

In [4]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.001, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.001, 0.001, 0.001, 0.001, 0.001,

In [5]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [6]:
dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   
display(agg.Simulation.Neighbor.idx_sum)
display(agg.Simulation.Neighbor.idx_red)

1×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 9  10  9  9  11  12  12  10  9  12  …  12  10  9  11  12  12  10  9  10  9

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  0   0   0   0  59  44  45   0   0  …  2462  2463     0     0     0     0
  0   0   0   0   0  60  61   0   0     2445  2446     0     0     0     0
  0   0   0   0   0   0   0   0   0        0    

# Sum Forces

In [8]:
threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(0.1),
        pi/3,
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)
display(agg.Position)

  2.493938 seconds (2.62 M CPU allocations: 172.527 MiB, 3.51% gc time)


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.000626234  -0.00467133   -0.00277445
  0.0034037    -0.00222225   -0.00310171
  0.00372308   -3.55103f-5   -0.00187945
  0.00133489    0.000822262  -0.00473754
  0.00485888   -0.00156919   -0.00313317
 -0.00262288    0.000897761   0.00534295
  0.00137278    0.00511903    0.00191356
 -0.0017857     0.00552607   -0.00236329
 -0.00215859    0.00139921    0.00292971
 -0.00442171    0.00153486   -0.00562478
 -0.00580766    0.0016181    -0.00319354
 -0.00382257    0.00447777    0.000245941
 -0.000989015  -0.00573164   -0.00231611
  ⋮                          
 -0.00357179   -0.00220504   -0.00222989
 -0.00161082    0.00553813    0.00169546
  0.00609809    0.00259497    0.00189687
 -0.000375798   0.00503645    0.00130884
 -0.00174308    0.00167925    0.0053075
  0.00290555    0.00412411    0.00480808
  0.00590847    0.0010034    -0.00156365
 -0.00144504    0.00622333    0.00265817
  0.000175885  -0.00385496    0.00457723
 -0.00246103    0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.50031   -4.62234  -13.8814
  0.501702  -4.62111  -13.8816
  2.50186   -4.62002  -13.8809
 -4.49933   -2.88959  -13.8824
 -2.49757   -2.89078  -13.8816
 -0.501311  -2.88955  -13.8773
  1.50069   -2.88744  -13.879
  3.49911   -2.88724  -13.8812
 -5.50108   -1.1493   -13.8785
 -3.50221   -1.14923  -13.8828
 -1.5029    -1.14919  -13.8816
  0.498089  -1.14776  -13.8799
  2.49951   -1.15287  -13.8812
  ⋮                   
 -1.50179    1.1489    13.8789
  0.499195   1.15277   13.8808
  2.50305    1.1513    13.8809
  4.49981    1.15252   13.8807
 -4.50087    2.89084   13.8827
 -2.49855    2.89206   13.8824
 -0.497046   2.8905    13.8792
  1.49928    2.89311   13.8813
  3.50009    2.88807   13.8823
 -1.50123    4.62162   13.8822
  0.498979   4.61912   13.8829
  2.49864    4.62193   13.8809

# MiniTest

In [9]:
run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing...   0%|▏                                   |  ETA: 0:00:41

ERROR: a exception was thrown during kernel execution.
       Run Julia on debug level 2 for device stack traces.


LoadError: CUDA error: an illegal instruction was encountered (code 715, ERROR_ILLEGAL_INSTRUCTION)